In [1]:
import torch
import torchvision
import torchvision.models.segmentation.deeplabv3 as dlv3
import torchvision.transforms.functional as tf
import numpy as np
import data
import utils
from PIL import Image
from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

In [2]:
def show_prediction(input_image, pred_image, label_color_map, label_string_map):
    label_legend_col = []

    
    # for i in range(len(label_color_map)):
    #     color = label_color_map.values()[i] + [255]
    #     color = np.array(color, dtype=np.float32)
    #     color /= 255.0
    #     label_legend_col.append(color.tolist())


In [3]:
model_parameters_load_path = r'G:/My Drive/Master IVA/Master Thesis/Models/20230321_211512/deeplabv3_model.pt'
    
ipt_path = r'G:\My Drive\Master IVA\Master Thesis\Datasets\real\test\rgb\real_rgb_3.png'
tgt_path = r'G:\My Drive\Master IVA\Master Thesis\Datasets\real\test\semantic_segmentation_mapped\real_semantic_segmentation_3.png'

num_classes = len(data.SemanticLabelMapper.ID_TO_STRING['common'].keys())
infer_width_resize = 128
infer_height_resize = 128

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = dlv3.deeplabv3_resnet50(pretrained=False, progress=True, num_classes=num_classes-1)
model.load_state_dict(torch.load(model_parameters_load_path))
model.eval()
model.to(device)
print('Loaded model to device.')

c:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Loaded model to device.


In [5]:
pil_ipt = Image.open(f'{ipt_path}', mode='r')
transform = torchvision.transforms.Resize((infer_height_resize, infer_width_resize))
ipt = tf.to_tensor(np.array(pil_ipt).astype(np.float32))
ipt = transform(ipt).unsqueeze(0).to(device)

pil_tgt = Image.open(f'{tgt_path}', mode='r')
transform = torchvision.transforms.Resize((infer_height_resize, infer_width_resize))
tgt = tf.to_tensor(np.array(pil_tgt).astype(np.float32)).long()
tgt = transform(tgt)

In [6]:
ignore_label = 19
with torch.no_grad():
    input_image = pil_ipt.resize(size=(infer_width_resize, infer_width_resize), resample=Image.Resampling.BILINEAR)
    
    pred = model(ipt)['out']
    pred = pred.data.cpu()[0]
    pred = np.argmax(pred, axis=0).unsqueeze(0)
    
    miou, ious = utils.measure_performance(predictions=pred, targets=tgt, num_classes=num_classes, ignore_label=ignore_label)

    print(f'Mean IoU: {miou}')
    print(f'IoUs: {ious}')
    
    show_prediction(input_image=input_image, 
                    pred_image=pred.squeeze(0).numpy(), 
                    label_color_map=data.SemanticLabelMapper.ID_TO_COLOR['common'],
                    label_string_map=data.SemanticLabelMapper.ID_TO_STRING['common'])

Mean IoU: 0.03394677047082526
IoUs: [0.5866205020403116, 1.4142968466562504e-18, 4.467698288230007e-19, 2.2204460492503132e-17, 1.5105075164968116e-18, 1.982541115402065e-18, 2.0185873175002845e-18, 4.440892098500626e-18, 5.337610695313253e-20, 1.3061447348531253e-17, 0.05836813690536824, 2.362176648138631e-18, 1.586032892321652e-17, 9.83368489481981e-20, 1.2335811384723961e-17, 0, 5.551115123125783e-17, 0, 7.401486830834377e-17, 0]
